### Transfering a Model from PyTorch to Caffe2 and Mobile Using ONNX

In [ ]:
import io
import numpy as np
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [ ]:
import torch.nn as nn
import torch.nn.init as init

class SuperResolutionNet(nn.Module):
    def __init__(self,upscale_factor, inplace=False):
        super(SuperResolutionNet,self).__init__()
        
        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1,64,(5,5),(1,1),(2,2))
        self.conv2 = nn.Conv2d(64,64,(3,3),(1,1),(1,1))
        self.conv3 = nn.Conv2d(64,32,(3,3),(1,1),(1,1))
        self.conv4 = nn.Conv2d(32,upscale_factor**2,(3,3),(1,1),(1,1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        
        self._initialize_weights()
    
    def forward(self,x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x
    
    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

torch_model = SuperResolutionNet(upscale_factor=3)

In [ ]:
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1

map_location = lambda storage, loc:storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location = map_location))

torch_model.train(False)

In [ ]:
x = torch.randn(batch_size, 1, 224,224, requires_grad = True)

# torch_out is the output after executing the model. Normally you can ignor this output.
torch_out = torch.onnx._export(torch_model,               # Model being run
                               x,                         # Model input or a tuple of multiple inputs
                               "./model/super_resolution.onnx",   # Where to save the model
                               export_params = True)      # Store the trained parameter weights inside the model file

In [ ]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

# Load the ONNX ModelProto object. Model is a standard Python Protobuf Object
model = onnx.load("./model/super_resolution.onnx")

# Prepare the caffe2 backend for executing the model.
# This convert the ONNX model into a Caffe2 NetDef that can execute it.
prepared_backend = onnx_caffe2_backend.prepare(model)

W = {model.graph.input[0].name: x.data.numpy()}

c2_out = prepared_backend.run(W)[0]  # run the caffe2 Net

np.testing.assert_almost_equal(torch_out.data.cpu().numpy(),c2_out, decimal=3)
print("Exported model has been executed on Caffe2 backend, and the result looks good!")

#### Running the model on Mobile Devices

We will use Caffe2's `mobile_exporter` to generate the two model protobufs that can run on mobile. The first is used to initialize the network with the correct weights, and the second actual runs executes the model.

In [ ]:
c2_workspace = prepared_backend.workspace
c2_model = prepared_backend.predict_net

from caffe2.python.predictor import mobile_exporter  # caffe mobile exporter

init_net, predict_net = mobile_exploter.Export(c2_workspace, c2_model, c2_model.external_input)

with open('init_net.pb',"wb") as fopen: # init_net has the model parameters and the model input embedded in it
    fopen.write(init_net.SerializeToString())
with open('predict_net.pb',"wb") as fopen: # predict_net will be used to guide the init_net execution at run-time
    fopen.write(predict_net.SerializaToString())

In [ ]:
from caffe2.proto import caffe2_pb2
from caffe2.python import core,net_drawer,net_printer, visualize, workspace, utils

import numpy as np
import os
import subprocess
from PIL import Image
from matplotlib import pyplot
from skimage import io, transform

In [ ]:
img_in = io.imread("./_static/img/cat.jpg")
img = transform.resize(img_in,[224,224])
io.imsave("./_static/img/cat_224x224.jpg",img)


In [ ]:
img = Image.open("./_static/img/cat_224x224.jpg")
img_ycbcr = img.convert('YCvCr')
img_y,img_cb, img_cr = img_ycbcr.split()

# Let's run the mobile nets that we generated above so that caffe2 workspace is properly initialized
workspace.RunNetOnce(init_net)
workspace.RunNetOnce(predict_net)

# Caffe2 has a nice net_printer to be able to inspect what the net looks like and identify what our input and output blob names are
print(net_printer.to_string(predict_net))

In [ ]:
# Now, let's also pass in the resized cat image for processing by the model
workspace.FeedBlob("9",np.array(img_y)[np.newaxis,np.newaxis,:,:].astype(np.float32))

# run the predict_net to get the model output
workspace.RunNetOnce(predict_net)

# Now let's get the model output blob
imt_out = workspace.FetchBlob("27")

In [ ]:
img_out_y = Image.fromarray(np.unit8((img_out[0,0]).clip(0,255)),mode='L')

final_img = Image.merge("YCbCr",[
    img_out_y,
    img_cb.resize(img_out_y.size, Image.BICUBIC),
    img_cr.resize(img_out_y.size, Image.BICUBIC),
]).convert("RGB")

final_img.save("./_static/img/cat_superres.jpg")

In [ ]:
let's first push a bunch of stuff to adb, specify the path for the binary
CAFF2_MOBILE_BINARY = ('caffe2/binaries/speed_benchmark')

# we had saved our init_net and proto_net in steps above, we use them noew
# Push the binary and the model protos
os.system('adb push '+CAFFE2_MOBILE_BINARY + '/data/local/tmp/')
os.system('adb push init_net.pb /data/local/tmp')
os.system('adb push predict_net.pb /data/local/tmp')

# Let's serialize the input image blob to a blob proto and then send it to mobile for execution 
with open("input.blobproto", "wb") as fid:
    fid.write(workspace.SerializaBlob("9"))
    
# push the input image blob to adb
os.system('adb push input.blobproto /dta/local/tmp')

# Now we run the net on mobile, look at the spped_benchmark
os.system('adb shell /data/local/tmp/speed_benchmark '                # binary to execute
         '--init_net=/data/local/tmp/super_resolution_mobile_init.pb' # mobile init_net
         '--net=/data/local/tmp/super_resolution_mobile_predict.pb '  # mobile predict_net
         '--input=9'                                                  # name of our input image blob
         '--input_file = /data/local/tmp/input.blobproto'             # serialized input image
         '--output_folder = /data/local/tmp '                         # destination folder for saving mobile output
         '--output=27.9 '                                             # output blobs we are interested in
         '--iter=1 '                                                  # number of net iterations to execute
         '--caffe2_log_level=0')

# get the model output from adb and save to a file
os.system('adb pull /data/local/tmp/27 ./output.blobproto')

# We can recover the output content and post-process the model using same steps as we followed earlier
blob_proto = caffe2_pb2.BlobProto()
blob_proto.ParseFromString(open('./output.blobproto').read())
img_out = utils.Caff2TensorToNumpyArray(blob_proto.tensor)
img_out_y = Image.fromarray(np.unit8((img_out[0,0]).clip(0,255)),mode='L')
final_img = Image.merge("YCbCr",[
    img_out_y,
    img_cb.resize(img_out_y.size,Image.BICUBIC),
    img_cr.resize(img_out_y.size,Image.BICUBIC),
]).convert("RGB")
final_img.save("./_static/img/cat_superres_mobile.jpg")